In [1]:
# =============================================================================
# bin/run_model.py
# @author: Brian Kyanjo
# @date: 2024-09-24
# @description: This script runs the flowline model using the EnKF data assimilation scheme.
# =============================================================================

In [2]:
# import libraries ========
import os
import sys
import jax
import warnings
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.stats import norm, multivariate_normal

warnings.filterwarnings("ignore")
jax.config.update("jax_enable_x64", True) # Set the precision in JAX to use float64

In [3]:
# Add the parent and `src` directories to sys.path if needed
for path in [os.path.abspath('..'), os.path.abspath('../src')]:
    if path not in sys.path:
        sys.path.insert(0, path)

# Import necessary modules
from config.config_loader import ParamsLoader
from EnKF.python_enkf.enkf_class_python import EnsembleKalmanFilter
from utils.utils import UtilsFunctions
# from flowline_model.flowline_model import FlowlineModel
import flowline_model.flowline_model as flowline_model

config_path = os.path.join(os.getcwd(), '../config', 'params.yaml') 

In [4]:
# Initial setup from the config file and initial guess ---------------
params = ParamsLoader(config_path).get_params()
grid = params['grid']

# Create instances of the necessary classes
utils_functions = UtilsFunctions(params)

# Initial guess and steady state ====================
xg = 300e3 / params["xscale"]
hf = (-utils_functions.bed(xg * params["xscale"]) / params["hscale"]) / (1 - params["lambda"])
h  = 1 - (1 - hf) * grid["sigma"]
u  = 1.0 * (grid["sigma_elem"] ** (1 / 3)) + 1e-3
huxg_old = np.concatenate((h, u, [xg]))

# Run the flowline model ====================
Jf = flowline_model.Jac_calc(huxg_old,params,grid,utils_functions.bed,flowline_model.flowline) # Jacobian of the model

# solve the system of equations
solve_result = root(
    lambda varin: flowline_model.flowline(varin,huxg_old,params,grid,utils_functions.bed), 
    huxg_old, 
    jac=Jf,
    method="hybr",
    options={"maxfev": 1000},
)
huxg_out0 = solve_result.x # extract the solution
# huxg_out0


In [5]:
# True simulation ====================
params["NT"] = 150
params["TF"] = params["year"] * 150
params["dt"] = params["TF"] / params["NT"]
params["transient"] = 1
params["facemelt"] = np.linspace(5, 85, params["NT"] + 1) / params["year"]
fm_dist = np.random.normal(0, 20.0)
fm_truth = params["facemelt"] 
params["facemelt"] = fm_truth

# Redefine the class instances with the new parameters
utils_functions = UtilsFunctions(params)

huxg_out1 = flowline_model.flowline_run(huxg_out0, params, grid, utils_functions.bed, flowline_model.flowline)


# Wrong simulation ====================
fm_wrong = np.linspace(5, 45, params["NT"] + 1) / params["year"]
params["facemelt"] = np.linspace(5, 45, params["NT"] + 1) / params["year"]

# Redefine the class instances with the new parameters
utils_functions = UtilsFunctions(params)
# flowline_model  =  FlowlineModel(params,grid,utils_functions.bed)

huxg_out2 = flowline_model.flowline_run(huxg_out0, params, grid, utils_functions.bed, flowline_model.flowline)

# Plot the results ====================
ts = np.linspace(0, params["TF"] / params["year"], params["NT"] + 1)

# xg_truth and xg_wrong calculations
xg_truth = np.concatenate(([huxg_out0[2 * params["NX"]]], huxg_out1[2 * params["NX"], :])) * params["xscale"]
# xg_wrong = np.concatenate(([huxg_out0[2 * params["NX"]]], huxg_out2[2 * params["NX"], :])) * params["xscale"]

# Plotting the results
plt.plot(ts, xg_truth / 1e3, lw=3, color='black', label="truth")
# plt.plot(ts, xg_wrong / 1e3, lw=3, color='red', label="wrong")
plt.plot(ts, 250.0 * np.ones_like(ts), lw=1, color='black', linestyle='--', label="sill")

# Add labels and legend
plt.xlabel("Time (years)")
plt.ylabel("xg (km)")
plt.legend()
plt.show()


Step 1

Step 2

Step 3

Step 4

Step 5

Step 6

Step 7

Step 8

Step 9



In [7]:
# Set ensemble parameters ====================
statevec_init = np.concatenate((huxg_out0, [params["facemelt"][0] / params["uscale"]]))

# Dimension of model state
nd = statevec_init.shape[0]

# Number of ensemble members
N = 30

# Define the standard deviations for model, observation, and process noise
sig_model = 1e-1
sig_obs   = 1e-2
sig_Q     = 1e-2

# Initialize the Cov_model matrix
Cov_model = (sig_model**2) * np.eye(nd)

# Initialize the Q matrix
Q = (sig_Q**2) * np.eye(nd)

# Set model parameters for single time step runs
nt = params["NT"]
tfinal_sim = params["TF"]

ts = np.arange(0.0, params["NT"] + 1) * params["year"]

# Update parameters for a single time step run
params["NT"] = 1
params["TF"] = params["year"] * 1
params["dt"] = params["TF"] / params["NT"]
params["transient"] = 1
params["assim"] = True  

# Concatenate elements similar to Julia's vcat
statevec_sig = np.concatenate((grid["sigma_elem"], grid["sigma"], np.array([1, 1])))

taper = np.ones((statevec_sig.shape[0], statevec_sig.shape[0]))
taper[-1, -3] = 2  
taper[-3, -1] = 2  
taper[-1, -1] = 10  
taper[-2, -1] = 10  
taper[-1, -2] = 10  

# Generate synthetic observations of thickness from the "truth" simulation
ts_obs = np.arange(10.0, 140.0 + 1, 10.0) * params["year"]

# Find the indices of ts that match ts_obs (equivalent to findall(in(ts_obs), ts))
idx_obs = np.nonzero(np.isin(ts, ts_obs))[0]

# Define the observation noise distribution (equivalent to Normal(0, sig_obs) in Julia)
obs_dist = norm(loc=0, scale=sig_obs)

# Create virtual observations by vertically concatenating huxg_out1 and fm_truth with added noise
fm_truth_scaled = fm_truth[idx_obs] / params["uscale"]
huxg_virtual_obs = np.vstack((huxg_out1[:, idx_obs], fm_truth_scaled.T))

# Add random noise to the virtual observations
huxg_virtual_obs += obs_dist.rvs(size=huxg_virtual_obs.shape)

# Set the number of observations
params["m_obs"] = 10

# Initialize the Ensemble Kalman Filter ====================
statevec_bg = np.zeros((nd, nt + 1))        # Background state vector (ub)
statevec_ens_mean = np.zeros((nd, nt + 1))  # Ensemble mean state vector (ua)
mm_ens_mean = np.zeros((nd - 1, nt + 1))    # Ensemble mean minus one dimension (ua)
statevec_ens = np.zeros((nd, N))            # Individual ensemble members (uai)
statevec_ens_full = np.zeros((nd, N, nt + 1)) # Full ensemble for all timesteps (uae)

# Set initial conditions
statevec_bg[:, 0] = statevec_init
statevec_ens_mean[:, 0] = statevec_init

# Initialize the ensemble with perturbations
for i in range(N):
    # Sample from a multivariate normal distribution
    perturbed_state = multivariate_normal.rvs(mean=np.zeros(nd-1), cov=Cov_model[:-1, :-1])
    
    # Assign the perturbed state to the ensemble, excluding the last element
    statevec_ens[:-1, i] = statevec_init[:-1] + perturbed_state
    
    # Keep the last element unchanged
    statevec_ens[-1, i] = statevec_init[-1]

# Store the ensemble initialization for the first timestep
statevec_ens_full[:, :, 0] = statevec_ens

In [ ]:
# # Run the Flowline model with the EnKF ====================
# for k in range(nt):
#     params["tcurrent"] = k + 1
#     print(f"Step {k+1}\n")

#     # Forecast step
#     statevec_bg[:-1, k+1] = np.squeeze(flowline_model.run_flowline(statevec_bg[:-1, k], flowline_model.flowline))